# Structured outputs with Azure OpenAI GPT-4o 2024-08-06

The GPT-4o-2024-08-06 model is designed to perform a wide range of tasks with minimal cost and latency, making it perfect for applications that require fast, real-time text responses. With the introduction of JSON Structured Outputs, it delivers 100% reliability in evaluations, ensuring outputs perfectly match the defined output schemas. This innovation enhances the efficiency and accuracy of AI-powered applications across diverse use cases.

### Structured Outputs: Revolutionizing AI Development
What are Structured Outputs? Structured Outputs allow developers to define the exact output format using a JSON Schema, making it easier to generate data in the desired structure for various applications. This feature is ideal for seamless integration with other systems.

### Key Benefits for Developers
- Consistent Data Formats: Outputs adhere to a defined JSON Schema, minimizing the need for manual post-processing.
- Flexible Tool Integrations: Supports specific function signatures, enabling precise outputs for tool integrations.
- Streamlined Application Development: Simplifies the creation of data payloads, enhancing the overall user experience.

### Example Use Case
A customer support chatbot can use Structured Outputs to generate responses formatted specifically for logging and analytics. This approach allows for seamless data integration, simplifying the process of logging, analyzing, and taking action based on the data.

### Documentation
https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure

### Regions
Available for standard and global deployments in all US regions and Sweden Central.
This model adds support for structured outputs when used in conjunction with the new 2024-08-01-preview inference API.

In [1]:
import datetime
import json
import openai
import sys

from dotenv import load_dotenv
from enum import Enum
from openai import AzureOpenAI
from pydantic import BaseModel
from typing import Union

In [2]:
print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai.__version__}")

Python version: 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
OpenAI version: 1.43.0


In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 06-Sep-2024 08:31:49


## Azure OpenAI

In [4]:
load_dotenv("azure.env")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT: str = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_KEY: str = os.getenv("AZURE_OPENAI_API_KEY")

In [5]:
MODEL:str = "gpt-4o-2024-08-06" # Your Azure OpenAI model
API_VERSION:str = "2024-08-01-preview" # Subject to change in the future

In [6]:
# Azure OpenAI client
client = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT,
                     api_key=AZURE_OPENAI_API_KEY,
                     api_version=API_VERSION)

## Example 1

In [7]:
# Structured outputs

class GetInformations(BaseModel):
    weather_location: str
    country_location: str
    country_language: str


tools = [openai.pydantic_function_tool(GetInformations)]

In [8]:
messages = []

messages.append({
    "role": "system",
    "content": "You are an AI helpful assistant."
})

messages.append({
    "role": "user",
    "content": "Hello, what is the current weather for the capital of France?"
})

In [9]:
response = client.chat.completions.create(model=MODEL,
                                          messages=messages,
                                          tools=tools)

In [10]:
response

ChatCompletion(id='chatcmpl-A4OfdQw8pj2NiO8jMnp3qTglw2b7D', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GRryPcMz0VNyPxjWOc2lwtyU', function=Function(arguments='{"weather_location":"Paris","country_location":"France","country_language":"French"}', name='GetInformations'), type='function')]))], created=1725611509, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=25, prompt_tokens=82, total_tokens=107))

In [11]:
print("\033[1;34m", end="")
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"weather_location":"Paris","country_location":"France","country_language":"French"}', name='GetInformations')


In [12]:
print("\033[1;34m", end="")
print(response.choices[0].message.tool_calls[0].function.arguments)

{"weather_location":"Paris","country_location":"France","country_language":"French"}


In [13]:
results = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

print("\033[1;34m", end="")
print(f"Location: {results['weather_location']}")
print(f"Country: {results['country_location']}")
print(f"Language: {results['country_language']}")

Location: Paris
Country: France
Language: French


In [14]:
print("\033[1;34m", end="")
print(response.model_dump_json(indent=3))

{
   "id": "chatcmpl-A4OfdQw8pj2NiO8jMnp3qTglw2b7D",
   "choices": [
      {
         "finish_reason": "tool_calls",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": null,
            "refusal": null,
            "role": "assistant",
            "function_call": null,
            "tool_calls": [
               {
                  "id": "call_GRryPcMz0VNyPxjWOc2lwtyU",
                  "function": {
                     "arguments": "{\"weather_location\":\"Paris\",\"country_location\":\"France\",\"country_language\":\"French\"}",
                     "name": "GetInformations"
                  },
                  "type": "function"
               }
            ]
         }
      }
   ],
   "created": 1725611509,
   "model": "gpt-4o-2024-08-06",
   "object": "chat.completion",
   "service_tier": null,
   "system_fingerprint": "fp_b2ffeb16ee",
   "usage": {
      "completion_tokens": 25,
      "prompt_tokens": 82,
      "total_tokens": 107
  

## Example 2

In [15]:
# Structured outputs

class ProductSupport(BaseModel):
    product: str
    brand: str
    issue: str


tools = [openai.pydantic_function_tool(ProductSupport)]

In [16]:
messages = []

messages.append({
    "role": "system",
    "content": "You are an AI helpful assistant."
})

messages.append({
    "role": "user",
    "content": "My Contoso headphones are not working anymore. No sound on the left."
})

In [17]:
response = client.chat.completions.create(model=MODEL,
                                          messages=messages,
                                          tools=tools)

In [18]:
response

ChatCompletion(id='chatcmpl-A4OfeX5Q5rZC6S2mfyiSF8m0D0SmM', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GRryPcMz0VNyPxjWOc2lwtyU', function=Function(arguments='{"product":"headphones","brand":"Contoso","issue":"No sound on the left"}', name='ProductSupport'), type='function')]))], created=1725611510, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=28, prompt_tokens=78, total_tokens=106))

In [19]:
print("\033[1;34m", end="")
print(response.choices[0].message.tool_calls[0].function.arguments)

{"product":"headphones","brand":"Contoso","issue":"No sound on the left"}


In [20]:
results = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

print("\033[1;34m", end="")
print(f"Product: {results['product']}")
print(f"Brand: {results['brand']}")
print(f"issue: {results['issue']}")

Product: headphones
Brand: Contoso
issue: No sound on the left


In [21]:
print("\033[1;34m", end="")
print(response.model_dump_json(indent=3))

{
   "id": "chatcmpl-A4OfeX5Q5rZC6S2mfyiSF8m0D0SmM",
   "choices": [
      {
         "finish_reason": "tool_calls",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": null,
            "refusal": null,
            "role": "assistant",
            "function_call": null,
            "tool_calls": [
               {
                  "id": "call_GRryPcMz0VNyPxjWOc2lwtyU",
                  "function": {
                     "arguments": "{\"product\":\"headphones\",\"brand\":\"Contoso\",\"issue\":\"No sound on the left\"}",
                     "name": "ProductSupport"
                  },
                  "type": "function"
               }
            ]
         }
      }
   ],
   "created": 1725611510,
   "model": "gpt-4o-2024-08-06",
   "object": "chat.completion",
   "service_tier": null,
   "system_fingerprint": "fp_b2ffeb16ee",
   "usage": {
      "completion_tokens": 28,
      "prompt_tokens": 78,
      "total_tokens": 106
   }
}


## Example 3

In [22]:
# Structured outputs

class CallCenter(BaseModel):
    travel_type: str
    from_city: str
    to_city: str
    travel_class:str
    travel_date:str
    from_country:str
    to_country:str
    airplane_company:str


tools = [openai.pydantic_function_tool(CallCenter)]

In [23]:
messages = []

messages.append({
    "role": "system",
    "content": "You are an AI helpful assistant."
})

messages.append({
    "role": "user",
    "content": "Hi, I need to purchase an Air France flight ticket from Paris to Seattle for Tuesday. \
I want a first class ticket."
})

In [24]:
response = client.chat.completions.create(model=MODEL,
                                          messages=messages,
                                          tools=tools)

In [25]:
response

ChatCompletion(id='chatcmpl-A4OfeJxhJTT2cAhlgq0WPSWwhZYJi', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_QzcRPtRB9FbgexZqy6vW6sS9', function=Function(arguments='{"travel_type":"Air","from_city":"Paris","to_city":"Seattle","travel_class":"First","travel_date":"2023-10-10","from_country":"France","to_country":"United States","airplane_company":"Air France"}', name='CallCenter'), type='function')]))], created=1725611510, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=58, prompt_tokens=136, total_tokens=194))

In [26]:
print("\033[1;34m", end="")
print(response.choices[0].message.tool_calls[0].function.arguments)

{"travel_type":"Air","from_city":"Paris","to_city":"Seattle","travel_class":"First","travel_date":"2023-10-10","from_country":"France","to_country":"United States","airplane_company":"Air France"}


In [27]:
print("\033[1;34m", end="")
print(response.model_dump_json(indent=3))

{
   "id": "chatcmpl-A4OfeJxhJTT2cAhlgq0WPSWwhZYJi",
   "choices": [
      {
         "finish_reason": "tool_calls",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": null,
            "refusal": null,
            "role": "assistant",
            "function_call": null,
            "tool_calls": [
               {
                  "id": "call_QzcRPtRB9FbgexZqy6vW6sS9",
                  "function": {
                     "arguments": "{\"travel_type\":\"Air\",\"from_city\":\"Paris\",\"to_city\":\"Seattle\",\"travel_class\":\"First\",\"travel_date\":\"2023-10-10\",\"from_country\":\"France\",\"to_country\":\"United States\",\"airplane_company\":\"Air France\"}",
                     "name": "CallCenter"
                  },
                  "type": "function"
               }
            ]
         }
      }
   ],
   "created": 1725611510,
   "model": "gpt-4o-2024-08-06",
   "object": "chat.completion",
   "service_tier": null,
   "system_fing